<a href="https://colab.research.google.com/github/AliAkbarBadri/topics-for-types/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [74]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 

In [75]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# EDA

In [77]:
questions = pd.read_csv("/content/drive/My Drive/topics-for-types/result.csv", delimiter=";")

In [78]:
print(questions.shape)
questions.head()

(2799, 21)


,id,title,sentence,subject1,subject2,subject3,subject_notsure,suggested_subject,type1,type2,type3,type_notsure,suggested_type,s1 score,s2 score,s3 score,t1 score,t2 score,t3 score,Avg_sbj_num,Avg_typ_num
0,1,NaN,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,ماشین,فنی,خرید,0,NaN,توضیح,تعریف,نظر,0,NaN,29,28,8,19,10,8,2.333333,1.333333
1,2,NaN,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,ماشین,فنی,کار,0,NaN,بله/خیر,نظر,توضیح,0,NaN,29,28,8,30,9,8,2.333333,1.666667
2,3,NaN,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,ماشین,فنی,کار,0,NaN,دلیل,نظر,توضیح,0,NaN,29,28,8,30,17,9,2.333333,2.000000
3,4,NaN,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,ماشین,فنی,کار,0,NaN,دلیل,توضیح,نظر,0,NaN,29,28,8,30,18,16,2.333333,2.333333
4,5,NaN,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,ماشین,فنی,خرید,0,NaN,نظر,لیست,تعریف,0,NaN,30,18,8,17,10,10,2.333333,2.333333


In [79]:
subjects_unique = len(set(np.concatenate((questions.subject1, questions.subject2,questions.subject3), axis=None)))
types_unique = len(set(np.concatenate((questions.type1, questions.type2,questions.type3), axis=None)))
print("number of subjects: " + str(subjects_unique) + ", number of types: " + str(types_unique))

number of subjects: 24, number of types: 13


In [80]:
chosen_columns = ['sentence','subject1',	'subject2',	'subject3',  'type1',	'type2',	'type3']

In [81]:
df_questions = questions[chosen_columns]
df_questions.head()

,sentence,subject1,subject2,subject3,type1,type2,type3
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,ماشین,فنی,خرید,توضیح,تعریف,نظر
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,ماشین,فنی,کار,بله/خیر,نظر,توضیح
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,ماشین,فنی,کار,دلیل,نظر,توضیح
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,ماشین,فنی,کار,دلیل,توضیح,نظر
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,ماشین,فنی,خرید,نظر,لیست,تعریف


In [82]:
df_questions[['subject1',	'subject2',	'subject3']].isna().sum(axis=1)

0       0
1       0
2       0
3       0
4       0
       ..
2794    1
2795    3
2796    3
2797    3
2798    3
Length: 2799, dtype: int64

In [137]:
df_questions[['type1',	'type2',	'type3']].isna().sum(axis=1)

0       0
1       0
2       0
3       0
4       0
       ..
2794    1
2795    2
2796    2
2797    3
2798    1
Length: 2799, dtype: int64

# Convert topics and types to multiple columns

In [84]:
df_questions['topics'] = df_questions[['subject1',	'subject2',	'subject3']].apply(
    lambda x: '|'.join(x.astype(str)),
    axis=1
)
df_questions['types'] = df_questions[['type1',	'type2',	'type3']].apply(
    lambda x: '|'.join(x.astype(str)),
    axis=1
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [85]:
df_questions.head()

,sentence,subject1,subject2,subject3,type1,type2,type3,topics,types
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,ماشین,فنی,خرید,توضیح,تعریف,نظر,ماشین|فنی|خرید,توضیح|تعریف|نظر
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,ماشین,فنی,کار,بله/خیر,نظر,توضیح,ماشین|فنی|کار,بله/خیر|نظر|توضیح
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,ماشین,فنی,کار,دلیل,نظر,توضیح,ماشین|فنی|کار,دلیل|نظر|توضیح
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,ماشین,فنی,کار,دلیل,توضیح,نظر,ماشین|فنی|کار,دلیل|توضیح|نظر
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,ماشین,فنی,خرید,نظر,لیست,تعریف,ماشین|فنی|خرید,نظر|لیست|تعریف


In [86]:
df_questions['types'].str.get_dummies('|')

,nan,انتخاب,بله/خیر,ترجمه,تعامل,تعریف,توضیح,دلیل,ریاضی,لیست,نظر,واقعیت,پیشنهاد
0,0,0,0,0,0,1,1,0,0,0,1,0,0
1,0,0,1,0,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,1,1,0,0,1,0,0
3,0,0,0,0,0,0,1,1,0,0,1,0,0
4,0,0,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,1,0,0,0,0,0,1,0,0,1,0,0,0
2795,1,0,0,0,0,0,0,0,0,1,0,0,0
2796,1,0,1,0,0,0,0,0,0,0,0,0,0
2797,1,0,0,0,0,0,0,0,0,0,0,0,0


In [87]:
df_questions['topics'].str.get_dummies('|')

,nan,اقتصاد,بازی,برنامه‌نویسی,تحصیل,تغذیه,حقوق/قانون,حیوانات,خرید,روابط,روانشناسی,زبان,زیبایی,سرگرمی,سفر,سلامت,فنی,ماشین,مذهبی,ورزش,کار,کامپیوتر/موبایل,کودکان,گیاهان
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
2795,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2796,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2797,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [132]:
df_topics = pd.concat([df_questions['sentence'], df_questions['topics'].str.get_dummies('|')],  axis=1, sort=False)
df_topics = df_topics[df_topics[df_topics.columns[2:]].sum(axis=1)!=0]
df_topics.drop(columns=['nan'],inplace=True, axis=1)
df_topics.head()

,sentence,اقتصاد,بازی,برنامه‌نویسی,تحصیل,تغذیه,حقوق/قانون,حیوانات,خرید,روابط,روانشناسی,زبان,زیبایی,سرگرمی,سفر,سلامت,فنی,ماشین,مذهبی,ورزش,کار,کامپیوتر/موبایل,کودکان,گیاهان
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [144]:
df_types = pd.concat([df_questions['sentence'], df_questions['types'].str.get_dummies('|')],  axis=1, sort=False)
df_types = df_types[df_types[df_types.columns[2:]].sum(axis=1)!=0]
df_types.drop(columns=['nan'],inplace=True, axis=1)
df_types.head()

,sentence,انتخاب,بله/خیر,ترجمه,تعامل,تعریف,توضیح,دلیل,ریاضی,لیست,نظر,واقعیت,پیشنهاد
0,تفاوت ترمز عقب دیسکی و کاسه ای چیست؟,0,0,0,0,1,1,0,0,0,1,0,0
1,آیا ترمز ضدقفل ABS در همه چرخ ها عمل می کند؟,0,1,0,0,0,1,0,0,0,1,0,0
2,چرا باوجود نصب بودن ترمز ABS بر روی خودرو، ترم...,0,0,0,0,0,1,1,0,0,1,0,0
3,چرا با وجود نصب بودن Airbag بر روی خودرو، در ه...,0,0,0,0,0,1,1,0,0,1,0,0
4,کلاس های مختلف خودرو چگونه شناسایی می شوند؟,0,0,0,0,1,0,0,0,1,1,0,0


In [149]:
df_topics.to_csv("/content/drive/My Drive/topics-for-types/df_topics.csv", index=False)
df_types.to_csv("/content/drive/My Drive/topics-for-types/df_types.csv", index=False)